# Capstone Two: Data Wrangling #

### Step 1: Data Collection

- Import packages
- Read in FEMA Redacted Claims dataset, dated 03-31-2019

In [1]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [45]:
# Read in dataset into dataframe called claims
claims = pd.read_csv('data/openFEMA_claims20190331.csv')

### Step 2: Data Organization

- GitHub repository created
- Folder structure created, dataset stored in data folder

### Step 3: Data Definition

- Explore variables and entries of dataset
- Identify issues that need to be cleaned

In [46]:
claims.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2418007 entries, 0 to 2418006
Data columns (total 39 columns):
 #   Column                                      Non-Null Count    Dtype  
---  ------                                      --------------    -----  
 0   agriculturestructureindicator               153883 non-null   object 
 1   asofdate                                    2418007 non-null  object 
 2   basefloodelevation                          480501 non-null   float64
 3   basementenclosurecrawlspacetype             2417962 non-null  float64
 4   reportedcity                                2413501 non-null  object 
 5   condominiumindicator                        2359895 non-null  object 
 6   policycount                                 2417999 non-null  float64
 7   countycode                                  2411261 non-null  float64
 8   crsdiscount                                 2417999 non-null  float64
 9   dateofloss                                  2418007 non-n

### Step 4: Data Cleaning

- Go through columns based on datatype (Y/N indicators, categorical, numerical, datetime)
- Create NaN column for each variable to capture missing values
- Fill NaNs in original columns with most appropriate value (e.g. create "isnull" category for categoricals)

**Y/N Indicator Variables**

In [47]:
# Define function for handling missing values in Y/N indicator variables
def YN_fill(df, col_list):
    for col in col_list:
        if set(df[col].unique()) == set(['N', 'Y', np.nan]):
            df[col] = df[col].replace({'Y':0, 'N':1})
            df[col] = df[col].fillna(2)
            df[col] = df[col].astype(int)
        else:
            print('invalid column: ' + col)
            
# Create list of Y/N indicator variables
YN_vars = ['agriculturestructureindicator', 'elevatedbuildingindicator', 'houseworship', 'nonprofitindicator', 'postfirmconstructionindicator', 'smallbusinessindicatorbuilding', 'primaryresidence']

In [48]:
# Call function
YN_fill(claims, YN_vars)

invalid column: nonprofitindicator


In [49]:
# Fix invalid entries in nonprofitindicator
claims['nonprofitindicator'] = claims['nonprofitindicator'].replace('0', np.nan)

In [50]:
# Call function again on nonprofitindicator
YN_fill(claims, ['nonprofitindicator'])

**Category Indicator Variables**

In [51]:
# Define function to reclassify indicator categories by column, see datadictionary for definitions
def reclass(df, col_list, r_dict_list):
    for i in range(0,len(col_list)):
        df[col_list[i]] = df[col_list[i]].replace(r_dict_list[i]) 

# Create reclassify dicts for each variable (num of floors and occtype ok as is)
bdcst_d = {0.0:1, 1.0:2, 2.0:3, 3.0:4, 4.0:5}
ci_d = {'N':1, 'U':2, 'A':3, 'H': 4, 'L':5, 'T': 6}
crsd_d = {0.0: 10, 0.5: 9, 1.0:8, 1.5: 7, 2.0:6, 2.5:5, 3.0:4, 3.5:3, 4.0:2, 4.5:1}
eci_d = {1.0:1, 2.0:2, 3.0:3, 4.0:4, 'A':5, 'B':6, 'C':7, 'D':8, 'E':9}
loc_d = {'Lowest floor only above ground level (No basement/enclosure/crawlspace/subgrade crawlspace)':3, 'Lowest floor above ground level and higher floors (No basement/enclosure/crawlspace/subgrade crawlspace)':4, 'Basement/Enclosure/Crawlspace/Subgrade Crawlspace and above':2, 'Manufactured (mobile) home or travel trailer on foundation':6, 'Above ground level more than one full floor':5, 'Basement/Enclosure/Crawlspace/Subgrade Crawlspace only':1}
ot_d = {'*':np.nan, 0:np.nan}
rm_d = {'A': 10, 'B': 11, 'E':12, 'F':13, 'G':14, 'P':15, 'Q':16, 'S':17, 'T':18, 'W':19}

# List of columns for reclassify
reclass_col = ['basementenclosurecrawlspacetype','condominiumindicator', 'crsdiscount', 'elevationcertificateindicator', 'locationofcontents', 'obstructiontype', 'ratemethod']

# List of dicts for reclassify
reclass_dict = [bdcst_d, ci_d, crsd_d, eci_d, loc_d, ot_d, rm_d]

In [52]:
# Call reclassify function
reclass(claims, reclass_col, reclass_dict)

In [53]:
# Turn elevationcertificateindicator and ratemethod to numeric
claims['elevationcertificateindicator'] = pd.to_numeric(claims['elevationcertificateindicator'])
claims['ratemethod'] = pd.to_numeric(claims['ratemethod'])

In [54]:
# Define function for creating new column to track NaNs for indicator variables, fill NaNs with 0
def create_nancol(df, col_list):
    for col in col_list:
        df[col+'_NaN'] = np.where(np.isnan(df[col].values), 1, 0)
        df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int)
        
# Create list of indicator variables 
ind_vars = ['basementenclosurecrawlspacetype','condominiumindicator', 'crsdiscount', 'elevationcertificateindicator', 'locationofcontents', 'numberoffloorsintheinsuredbuilding', 'obstructiontype', 'occupancytype', 'ratemethod']

In [55]:
# Call function
create_nancol(claims, ind_vars)

In [56]:
# Address floodzone, keep categories as strings, fill NaNs with 0
claims['floodzone_NaN'] = claims['floodzone'].isnull().astype(int)
claims['floodzone'] = claims['floodzone'].fillna('isnull')

**Continuous Numerical Variables**

In [57]:
# Create NaN column function on numerical variables
# Also fill with mode
def create_nancol_num(df, col_list):
    for col in col_list:
        df[col+'_NaN'] = np.where(np.isnan(df[col].values), 1, 0)
        df[col] = df[col].fillna(df[col].mode()[0])

# Create list of applicable columns
num_vars = ['policycount', 'lowestadjacentgrade', 'lowestfloorelevation','amountpaidonbuildingclaim', 'amountpaidoncontentsclaim', 'amountpaidonincreasedcostofcomplianceclaim', 'totalbuildinginsurancecoverage', 'totalcontentsinsurancecoverage', 'elevationdifference', 'basefloodelevation']

In [58]:
# Call function
create_nancol_num(claims, num_vars)

**Date Variables**

In [59]:
# Address invalid date entry in originalconstructiondate
claims['originalconstructiondate'] = claims['originalconstructiondate'].replace('1111-11-11', np.nan)

In [60]:
# Create function to create NaN column and turn date columns into datetime type
def create_nancol_date(df, col_list):
    for col in col_list:
        df[col+'_NaN'] = df[col].apply(lambda x: 1 if x == np.nan else 0)
        df[col] = pd.to_datetime(df[col])
        
# Create list of applicable columns
date_vars = ['asofdate', 'dateofloss', 'originalconstructiondate', 'originalnbdate']

In [61]:
# Call function
create_nancol_date(claims, date_vars)

**Other Cat Variables**

In [62]:
# Define function to create NaN column and fill NaNs with "isnull"
def create_nancol_ocat(df, col_list):
    for col in col_list:
        df[col+'_NaN'] = df[col].isnull().astype(int)
        df[col] = df[col].fillna('isnull')
        
# Create list of applicable columns
ocat_vars = ['reportedcity', 'countycode', 'censustract', 'latitude', 'longitude', 'yearofloss', 'state']

In [63]:
# Call function
create_nancol_ocat(claims, ocat_vars)

In [64]:
# Address reportedzip
# Replace invalid entries
create_nancol_ocat(claims, ['reportedzip'])
claims['reportedzip'] = claims['reportedzip'].replace(to_replace=r'^\D\d{5}\D', value='invalid', regex=True)

**Output cleaned dataset for next steps**

In [65]:
claims.to_csv('data/claimsdata_clean.csv', index=False)